In [ ]:
import flwr as fl
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
import os
import random

# Read method and number of rounds from config.txt
if os.path.exists("config.txt"):
    with open("config.txt", "r") as f:
        lines = f.readlines()
        method = lines[0].strip()
        num_rounds = int(lines[1].strip())
        dataset = lines[2].strip()
        seed = lines[3].strip()
else:
    raise FileNotFoundError("Config file not found. Please run the server first.")

if dataset == "Noisy":
    train_df = pd.read_csv(r".\trainWNoise.csv")
    test_df = pd.read_csv(r".\testWNoise.csv")
else:
    train_df = pd.read_csv(r".\Train_Aggregated.csv")
    test_df = pd.read_csv(r".\Test_Aggregated.csv")

# Parameters
data_per_client = 247  # Training data points per client.
test_data_points = 102  # Number of test data points.
sequence_length = 50
prediction_length = 50
num_iterations = 10

# Initialize storage for results
client_loss_data_all = {}
first_seed = seed

# Calculate the number of clients
train_data_size = len(train_df)
num_clients = train_data_size // data_per_client

# Verify that we have enough data
total_required_data = num_clients * data_per_client
if train_data_size < total_required_data:
    raise ValueError(f"Not enough training data. Required: {total_required_data}, Available: {train_data_size}")

# Ask the user if they want to change the method
default_method = method # Set the default method
method_map = {"0": "FedAvg", "1": "FedDyn", "2": "IDA", "3": "Robust"}
method_input = input(f"Enter the training method (0: FedAvg, 1: FedDyn, 2: IDA, 3: Robust) or press Enter to keep [{default_method}]: ").strip()

# If the user provided a method, update it; otherwise, keep the default
if method_input in method_map:
    method = method_map[method_input]
else:
    method = default_method
print(f"Using method: {method}")  # Confirm the method selection
    
random.seed(seed) 

device_evaluation_df = pd.DataFrame(columns=["client_id", "loss"])
client_loss_data = {}

def dynamic_weight_decay(loss, base_decay=0.01, min_decay=0.0001, max_decay=0.1):
    """
    Adjust weight decay based on the loss.
    Higher loss leads to higher regularization (to prevent overfitting).
    Lower loss leads to lower regularization (to allow better fitting).
    
    Args:
        loss (float): Current loss value.
        base_decay (float): Base weight decay value.
        min_decay (float): Minimum weight decay allowed.
        max_decay (float): Maximum weight decay allowed.
        
    Returns:
        float: Adjusted weight decay value.
    """
    # Adjust weight decay inversely with the loss (clipped to min/max bounds)
    decay = base_decay * (1 + loss)
    decay = np.clip(decay, min_decay, max_decay)
    return decay

def inverse_distance_aggregation(loss, base_decay=0.01, min_decay=0.0001, max_decay=0.1):
    """
    Adjust weight decay based on the loss using Inverse Distance Aggregation (IDA) and INTRAC.
    
    Args:
        loss (float): Current loss value.
        base_decay (float): Base weight decay value.
        min_decay (float): Minimum weight decay allowed.
        max_decay (float): Maximum weight decay allowed.
        
    Returns:
        float: Adjusted weight decay value.
    """
    alpha_k = 1 / (abs(loss) + 1e-8)  # Inverse distance metric
    alpha_k = np.clip(alpha_k, 1e-3, 10)  # Clamping values for stability
    decay = base_decay * alpha_k
    decay = np.clip(decay, min_decay, max_decay)
    return decay

def one_step_weiszfeld(losses, base_decay=0.01, min_decay=0.0001, max_decay=0.1):
    """
    Adjust weight decay using a modified Weiszfeld algorithm.

    Args:
        losses (list or np.array): List of client loss values.
        base_decay (float): Base weight decay value.
        min_decay (float): Minimum weight decay allowed.
        max_decay (float): Maximum weight decay allowed.

    Returns:
        float: Adjusted weight decay value.
    """
    losses = np.array(losses)
    
    # Compute the simple arithmetic mean
    mean_loss = np.mean(losses)
    
    # Compute the weighted consensus point using one-step Weiszfeld iteration
    weights = 1 / (np.abs(losses) + 1e-8)  # Avoid division by zero
    weights /= np.sum(weights)  # Normalize weights
    consensus_point = np.sum(weights * losses)

    # Compute decay as Euclidean distance between consensus point and arithmetic mean
    decay = np.abs(consensus_point - mean_loss)
    
    # Scale the decay using the base decay factor
    decay = base_decay * decay
    decay = np.clip(decay, min_decay, max_decay)

    return decay

# Custom Dataset for Time Series Data
class TimeSeriesDataset(Dataset):
    def __init__(self, data, sequence_length=sequence_length, prediction_length=prediction_length):
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length
        self.data = data.reset_index(drop=True)
        self.X, self.y = self._prepare_data()

    def _prepare_data(self):
        data_array = self.data.values
        X, y = [], []
        for i in range(len(data_array) - self.sequence_length - self.prediction_length + 1):
            X.append(data_array[i:i + self.sequence_length])
            y.append(
                data_array[
                    i + self.sequence_length : i + self.sequence_length + self.prediction_length, -1
                ]
            )  # Assuming target is last column
        X = np.array(X)
        y = np.array(y)
        return X, y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.y[idx], dtype=torch.float32)
        return X, y

# LSTM Model Definition
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, prediction_length):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.prediction_length = prediction_length
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True
        )
        self.fc = nn.Linear(hidden_size, prediction_length)

    def forward(self, x):
        h0 = torch.zeros(
            self.num_layers, x.size(0), self.hidden_size
        ).to(x.device)
        c0 = torch.zeros(
            self.num_layers, x.size(0), self.hidden_size
        ).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]  # Take the last output
        out = self.fc(out)
        return out

# Federated Learning Client
class FLClient(fl.client.NumPyClient):
    def __init__(self, client_data):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.sequence_length = sequence_length
        self.prediction_length = prediction_length
        self.input_size = client_data.shape[1]  # Number of features
        self.hidden_size = 64
        self.num_layers = 2

        # Initialize the model
        self.model = LSTMModel(
            input_size=self.input_size,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            prediction_length=self.prediction_length,
        ).to(self.device)

        # Prepare datasets and dataloaders
        self.train_dataset = TimeSeriesDataset(client_data, self.sequence_length, self.prediction_length)
        self.train_loader = DataLoader(self.train_dataset, batch_size=32, shuffle=True)

        # Define loss function (MSE)
        self.criterion = nn.MSELoss()
        self.base_weight_decay = 0.01  # Base value for weight decay
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        params_dict = zip(self.model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(np.copy(v), dtype=torch.float32) for k, v in params_dict}
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        epoch_loss = 0.0

        # Adjust weight decay based on the current performance
        if method == "FedDyn":
            current_weight_decay = dynamic_weight_decay(epoch_loss)
        elif method == "IDA":
            current_weight_decay = inverse_distance_aggregation(epoch_loss)
        elif method == "Robust":
            current_weight_decay = one_step_weiszfeld(epoch_loss)
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001, weight_decay=current_weight_decay)

        for epoch in range(1):  # Single epoch per round
            for X_batch, y_batch in self.train_loader:
                X_batch = X_batch.to(self.device)
                y_batch = y_batch.to(self.device)
                self.optimizer.zero_grad()
                output = self.model(X_batch)
                loss = self.criterion(output, y_batch)
                epoch_loss += loss.item()
                loss.backward()
                self.optimizer.step()

        # Return the updated model parameters
        return self.get_parameters(config), len(self.train_loader.dataset), {"weight_decay": current_weight_decay}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        loss = 0.0
        total = 0
        with torch.no_grad():
            for X_batch, y_batch in self.train_loader:
                X_batch = X_batch.to(self.device)
                y_batch = y_batch.to(self.device)
                output = self.model(X_batch)
                loss += self.criterion(output, y_batch).item() * X_batch.size(0)
                total += X_batch.size(0)
        loss /= total
        return float(loss), total, {"MSE": float(loss)}

# Function to create client instances
def client_fn(cid: str):
    # Extract client ID from cid
    client_id = int(cid)

    # Validate client_id
    if client_id >= num_clients:
        raise ValueError(f"Invalid client_id {client_id}. It should be less than {num_clients}.")

    # Calculate the start and end indices for this client
    start_idx = (client_id  + random.randint(1,30))* data_per_client
    end_idx = start_idx + data_per_client

    #loop around if too big
    if start_idx > len(train_df):
        start_idx = start_idx - len(train_df)
        end_idx = start_idx + data_per_client


    # Ensure we don't exceed the dataset length
    if end_idx > len(train_df):
        end_idx = len(train_df)

    # Slice the data for this client
    client_data = train_df.iloc[start_idx:end_idx].reset_index(drop=True)

    # Handle the case where there might be less data than expected
    if len(client_data) < data_per_client:
        print(f"Client {client_id} has {len(client_data)} data points (less than expected).")

    # Debugging output
    print(f"Client {client_id} has {len(client_data)} training data points.")

    # Create and return a client instance with this data
    return FLClient(client_data)

# Prepare test data
test_data_size = len(test_df)
if test_data_size >= test_data_points:
    test_data = test_df.iloc[:test_data_points].reset_index(drop=True)
else:
    raise ValueError(f"Not enough test data. Required: {test_data_points}, Available: {test_data_size}")

# Define a function for server-side evaluation
def get_evaluate_fn(model):
    def evaluate(server_round, parameters, config):
        global client_loss_data

        # Update the model with the current global parameters
        params_dict = zip(model.state_dict().keys(), parameters)
        state_dict = {k: torch.tensor(np.copy(v), dtype=torch.float32) for k, v in params_dict}
        model.load_state_dict(state_dict, strict=True)

        # Simulate loss evaluation for each client
        model.eval()
        for client_id in range(num_clients):  # Iterate over all possible clients
            try:
                client = client_fn(str(client_id))  # Get the client
                client_loss = 0.0
                total_samples = 0

                # Evaluate the client's model on its own dataset
                with torch.no_grad():
                    for X_batch, y_batch in client.train_loader:
                        X_batch = X_batch.to(model.device)
                        y_batch = y_batch.to(model.device)
                        output = model(X_batch)
                        client_loss += nn.MSELoss()(output, y_batch).item() * X_batch.size(0)
                        total_samples += X_batch.size(0)

                # Compute the average loss for this round
                if total_samples > 0:
                    round_loss = client_loss / total_samples
                else:
                    round_loss = float("nan")

                # Update the cumulative loss data
                if client_id not in client_loss_data:
                    client_loss_data[client_id] = {"total_loss": 0.0, "rounds": 0}
                client_loss_data[client_id]["total_loss"] += round_loss
                client_loss_data[client_id]["rounds"] += 1

            except ValueError as e:
                # If a client does not exist (not selected), skip it
                print(f"Client {client_id} not included in this round: {e}")

        return None, {}  # No need to return global metrics since it's per-client loss
    return evaluate

# After the simulation, save the averaged client losses to a CSV file
def save_averaged_device_losses(filename="averaged_device_losses.csv"):
    global client_loss_data

    # Compute the average loss for each client
    averaged_data = []
    for client_id, data in client_loss_data.items():
        avg_loss = data["total_loss"] / data["rounds"] if data["rounds"] > 0 else float("nan")
        averaged_data.append({"client_id": client_id, "average_loss": avg_loss})

    # Create a DataFrame and save it to CSV
    averaged_df = pd.DataFrame(averaged_data)
    averaged_df.to_csv(filename, index=False)

# Loop for 50 iterations, changing the seed each time
for i in range(num_iterations):
    seed = i + 1  # Change seed in each run
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    # Reset global loss storage
    client_loss_data = {}

    # Initialize the global model
    global_model = LSTMModel(
        input_size=train_df.shape[1],
        hidden_size=64,
        num_layers=2,
        prediction_length=prediction_length
    )
    global_model.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    global_model.to(global_model.device)

    # Start simulation
    fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=num_rounds),
        strategy=fl.server.strategy.FedAvg(
            evaluate_fn=get_evaluate_fn(global_model)
        ),
    )

    # Compute average loss for each client
    averaged_data = {}
    for client_id, data in client_loss_data.items():
        avg_loss = data["total_loss"] / data["rounds"] if data["rounds"] > 0 else float("nan")
        averaged_data[client_id] = avg_loss

    # Store results in a dictionary
    for client_id in range(num_clients):
        if client_id not in client_loss_data_all:
            client_loss_data_all[client_id] = []
        client_loss_data_all[client_id].append(averaged_data.get(client_id, float("nan")))

# Convert to DataFrame
df = pd.DataFrame.from_dict(client_loss_data_all, orient="index")
df.insert(0, "client_id", df.index)  # Insert client_id column
df.columns = ["client_id"] + [f"average_loss in {str(int(first_seed)+i)}" for i in range(num_iterations)]  # Name columns

# Save results
output_filename = f"ANOVA_averaged_device_losses_{method}_{dataset}.csv"
df.to_csv(output_filename, index=False)

print(f"Results saved to {output_filename}")

Using method: FedAvg


INFO :      Starting Flower simulation, config: num_rounds=100, no round_timeout
2025-03-04 19:56:28,833	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 16.0, 'accelerator_type:G': 1.0, 'memory': 5710174619.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 2855087308.0, 'GPU': 1.0, 'node:__internal_head__': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      No `client_resources` specified. Using minimal resources for clients.
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 0.0}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 16 actors
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
(pid=17404) 2025-03-04 19:56:32.417222: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to flo

(ClientAppActor pid=17484) Client 7 has 247 training data points.


(ClientAppActor pid=17484) c:\Users\McZom\anaconda3\envs\TestCuda\Lib\site-packages\botocore\utils.py:15: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
(ClientAppActor pid=17484)   import cgi
(ClientAppActor pid=17484) c:\Users\McZom\anaconda3\envs\TestCuda\Lib\site-packages\botocore\httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
(ClientAppActor pid=17484)   from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext
(ClientAppActor pid=17484) 2025-03-04 19:56:42.425128: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(ClientAppActor pid=17484) To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.

Client 0 has 247 training data points.


...

ERROR :     ray::ClientAppActor.run() (pid=17484, ip=127.0.0.1, actor_id=943e5761189a705d5c05551c01000000, repr=<flwr.simulation.ray_transport.ray_actor.ClientAppActor object at 0x0000022374C03490>)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\McZom\anaconda3\envs\TestCuda\Lib\site-packages\flwr\client\client_app.py", line 143, in __call__
    return self._call(message, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\McZom\anaconda3\envs\TestCuda\Lib\site-packages\flwr\client\client_app.py", line 126, in ffn
    out_message = handle_legacy_message_from_msgtype(
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\McZom\anaconda3\envs\TestCuda\Lib\site-packages\flwr\client\message_handler\message_handler.py", line 129, in handle_legacy_message_from_msgtype
    fit_res = maybe_call_fit(
              ^^^^^^^^^^^^^^^
  File "c:\Users\McZom\anaconda3\envs\TestCuda\Lib\site-packages\flwr\client\client.py", line 255, in maybe_ca

(ClientAppActor pid=17484) Client 0 has 247 training data points.


(ClientAppActor pid=17456) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=17456) 
(ClientAppActor pid=17456)             This is a deprecated feature. It will be removed
(ClientAppActor pid=17456)             entirely in future versions of Flower.
(ClientAppActor pid=17456)         


(ClientAppActor pid=17456) Client 9 has 247 training data points.


(ClientAppActor pid=17456) c:\Users\McZom\anaconda3\envs\TestCuda\Lib\site-packages\botocore\utils.py:15: DeprecationWarning: 'cgi' is deprecated and slated for removal in Python 3.13
(ClientAppActor pid=17456)   import cgi
(ClientAppActor pid=17456) c:\Users\McZom\anaconda3\envs\TestCuda\Lib\site-packages\botocore\httpsession.py:41: DeprecationWarning: 'urllib3.contrib.pyopenssl' module is deprecated and will be removed in a future release of urllib3 2.x. Read more in this issue: https://github.com/urllib3/urllib3/issues/2680
(ClientAppActor pid=17456)   from urllib3.contrib.pyopenssl import orig_util_SSLContext as SSLContext
(ClientAppActor pid=17456) 2025-03-04 19:56:52.217961: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
(ClientAppActor pid=17456) To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.